****
# Excess Workshop Data Visualization
****

Contact Information

... Abstract ...

## Introduction

... introduction about the excess workshop ...

In case you have not installed all libraries, the following cell will do the installation.

In [1]:
!pip install numpy matplotlib ipywidgets

You should consider upgrading via the '/Users/felix/.pyenv/versions/3.8.6/bin/python3.8 -m pip install --upgrade pip' command.


Import all libraries.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
%config InlineBackend.figure_formats = ['svg']

## Import Data

Import the energy values.

In [3]:
exp_A = np.loadtxt('test_data/experiment_A.txt')
exp_B = np.loadtxt('test_data/experiment_B.txt')
exp_C = np.loadtxt('test_data/experiment_C.txt')

Import the efficiency curves.

In [4]:
eff_A = np.loadtxt('test_data/experiment_A_eff.txt')
eff_B = np.loadtxt('test_data/experiment_B_eff.txt')
eff_C = np.loadtxt('test_data/experiment_C_eff.txt')

Define the exposure of all experiments.

In [5]:
exposure_A = 1
exposure_B = 0.2
exposure_C = 15

## Plot

Here we do the interactive plot.

In [6]:
@interact(bins = widgets.IntSlider(value=500,min=20,max=2000),
          x_range = widgets.FloatRangeSlider(value=[0,20],min=0,max=20),
          y_max = widgets.FloatLogSlider(value=10000,min=-1,max=5),
          check_A = widgets.Checkbox(value=True, description='Plot Experiment A'),
          check_B = widgets.Checkbox(value=True, description='Plot Experiment B'),
          check_C = widgets.Checkbox(value=True, description='Plot Experiment C'),
          scale_drop = widgets.Dropdown(options=['linear', 'log'], value='linear', description='Scale: ',),
          shift_A = widgets.FloatSlider(value=0,min=-20,max=20),
          shift_B = widgets.FloatSlider(value=0,min=-20,max=20),
          shift_C = widgets.FloatSlider(value=0,min=-20,max=20),
          scale_A = widgets.FloatLogSlider(value=1,min=-10,max=10),
          scale_B = widgets.FloatLogSlider(value=1,min=-10,max=10),
          scale_C = widgets.FloatLogSlider(value=1,min=-10,max=10),
          offset_A = widgets.FloatSlider(value=0,min=-10000,max=10000),
          offset_B = widgets.FloatSlider(value=0,min=-10000,max=10000),
          offset_C = widgets.FloatSlider(value=0,min=-10000,max=10000),
          check_eff = widgets.Checkbox(value=True, description='Correct Efficiency'),
         )
def plot_all(bins, 
             x_range, 
             y_max, 
             check_A, 
             check_B, 
             check_C, 
             scale_drop,
             shift_A,
             shift_B,
             shift_C,
             scale_A,
             scale_B,
             scale_C,
             offset_A,
             offset_B,
             offset_C,
             check_eff,
            ):
    
    # shift and scale values
    experiment_A = exp_A + shift_A
    experiment_B = exp_B + shift_B
    experiment_C = exp_C + shift_C

    # make histograms

    hist_A, bins_A = np.histogram(experiment_A, bins, range=x_range)
    hist_B, bins_B = np.histogram(experiment_B, bins, range=x_range)
    hist_C, bins_C = np.histogram(experiment_C, bins, range=x_range)
    
    # reweight with efficiencies

    if check_eff:
        hist_A = hist_A / np.interp(bins_A[:-1], eff_A[:,0], eff_A[:,1])
        hist_B = hist_B / np.interp(bins_B[:-1], eff_B[:,0], eff_B[:,1])
        hist_C = hist_C / np.interp(bins_C[:-1], eff_C[:,0], eff_C[:,1])
    
    # define the weights
    
    weights_A = scale_A*hist_A/exposure_A + offset_A
    weights_B = scale_B*hist_B/exposure_B + offset_B
    weights_C = scale_C*hist_C/exposure_C + offset_C
    
    # all negative weights are zero
    
    weights_A[weights_A < 0] = 0
    weights_B[weights_B < 0] = 0
    weights_C[weights_C < 0] = 0

    # plot - comment the lines of experiments to not show them

    if check_A:
        plt.hist(bins_A[:-1], bins_A, weights=weights_A, histtype='step', label='Experiment A', color='C0')
    if check_B:
        plt.hist(bins_B[:-1], bins_B, weights=weights_B, histtype='step', label='Experiment B', color='C1')
    if check_C:
        plt.hist(bins_C[:-1], bins_C, weights=weights_C, histtype='step', label='Experiment C', color='C2')
    plt.xlabel('Energy (keV)')
    plt.ylabel(f'Counts / ({1000*(x_range[1] - x_range[0])/bins:.1f} eV kg days)')
    plt.legend()
    plt.xlim(x_range)
    plt.ylim((0.5, y_max))
    plt.yscale(scale_drop)
    plt.show()

interactive(children=(IntSlider(value=500, description='bins', max=2000, min=20), FloatRangeSlider(value=(0.0,…

## Download Data

Coming soon...